# Code with face recognition

In [1]:
# imports
import cv2
import os
import numpy as np
from base64 import b64decode, b64encode
import PIL
import io
import html
import time
from statistics import mean
import torch.nn.functional as F
from torch import optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.transforms.functional  import InterpolationMode
import torchvision
from torchsummary import summary
from datetime import datetime
import threading

In [2]:
class liveClassifier(nn.Module):
    def __init__(self):
        super(liveClassifier, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        # Conv output = [(256+2x1-3)\1] + 1 = 256
        # Max pooling = (256\2) = 128
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2))
        # Conv output = [(128+2x0-3)\1] + 1 = 126
        # Max pooling = (126\2) = 63
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2))
        # Conv output = [(63+2x0-3)\1] + 1 = 61
        # Max pooling = (61\2) = 30
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2))
        # Conv output = [(30+2x0-3)\1] + 1 = 28
        # Max pooling = (28\2) = 14

        self.fc1 = nn.Linear(256 * 14 * 14, 1024)
        self.drop = nn.Dropout2d(0.5)
        self.fc2 = nn.Linear(1024, 512)
        self.drop = nn.Dropout2d(0.5)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 3)


    def forward(self, input):
        out = self.layer1(input)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.drop(out)
        out = self.fc3(out)
        out = self.fc4(out)
        return out


In [3]:
model = liveClassifier()
model.load_state_dict(torch.load(r'C:\Users\agran\OneDrive - Holon Institute of Technology\3. Year 3\Semecter B\מבוא לתוכנה משובצת מחשב\Python_Code_face_regoc\best_model_checkpoint_1.pt'))  # Load your trained model


<All keys matched successfully>

In [4]:
import cv2
import torch
import torch.nn.functional as F
from torchvision import transforms
import serial
import time
from collections import deque

# Open the serial port (replace 'COM5' with the correct port for your system)
ser = serial.Serial('COM5', 9600)
time.sleep(2)  # Wait for the serial connection to initialize

# Define the transform for the video frames
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define class names (adjust based on your dataset)
class_names = ['Sahar', 'Yahel', 'Others']

# Initialize video capture (0 for webcam or provide video file path)
cap = cv2.VideoCapture(0)

# Define the codec and create VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

# Settings for distance checking
threshold_distance = 50  # cm
consecutive_readings_required = 3  # Number of consecutive valid readings below the threshold
buffer_size = 5  # Size of the buffer for the moving average

# Initialize variables for distance checking
distance_buffer = deque(maxlen=buffer_size)
below_threshold_counter = 0
timeout_start = None
timeout_duration = 10  # seconds

while True:
    # Check distance from Arduino sensor
    line = ser.readline().decode('utf-8').strip()
    try:
        # Extract the distance from the line
        if line.startswith("Distance:"):
            parts = line.split(",")
            for part in parts:
                if "Distance" in part:
                    distance = int(part.split(":")[1].strip().replace("cm", ""))

                    # Print the distance
                    print("Distance:", distance, "cm")

                    # Update the distance buffer
                    distance_buffer.append(distance)

                    # Calculate the moving average distance
                    if len(distance_buffer) == buffer_size:
                        avg_distance = sum(distance_buffer) / buffer_size
                        print("Average Distance:", avg_distance, "cm")

                        # Check if the average distance is below the threshold
                        if avg_distance < threshold_distance:
                            below_threshold_counter += 1
                            if below_threshold_counter >= consecutive_readings_required:
                                timeout_start = None  # Reset timeout if object is within 50 cm
                                if not cap.isOpened():
                                    cap.open(0)

                                ret, frame = cap.read()
                                if not ret:
                                    break

                                frame = cv2.flip(frame, 1)

                                # Preprocess the frame
                                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                                img = cv2.resize(img, (256, 256))
                                img = transform(img).unsqueeze(0)

                                with torch.no_grad():
                                    outputs = model(img)
                                    probabilities = F.softmax(outputs, dim=1)
                                    confidence, predicted = torch.max(probabilities, 1)
                                    pred_class_name = class_names[predicted.item()]
                                    pred_confidence = confidence.item()

                                # Plot classification result on the frame
                                x, y = 50, 50  # Position to display the text
                                bbox_array = cv2.putText(frame, f"{pred_class_name}: {pred_confidence:.2f}", (x, y),
                                                         cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255, 0), 1)
                                bbox_array_1 = cv2.putText(frame, "Press 'q' to Exit", (400, 50),
                                                           cv2.FONT_HERSHEY_TRIPLEX, 0.6, (255, 255, 255, 0), 1)

                                # Send signal to Arduino based on the classification and confidence
                                if pred_class_name in ['Yahel']:
                                    ser.write(b'1')
                                    print("Sent: 1")
                                else:
                                    ser.write(b'0')
                                    print("Sent: 0")

                                # Display the frame
                                cv2.imshow('Frame', frame)

                                # Write the frame with the classification to the output video
                                out.write(frame)

                                # Break the loop on 'q' key press
                                if cv2.waitKey(1) & 0xFF == ord('q'):
                                    break

                        else:
                            below_threshold_counter = 0
                            if timeout_start is None:
                                timeout_start = time.time()  # Start timeout if object moves out of 50 cm

                            if timeout_start and time.time() - timeout_start >= timeout_duration:
                                print("Object not within 50 cm for 10 seconds. Pausing inference.")
                                if cap.isOpened():
                                    cap.release()
                                out.release()
                                cv2.destroyAllWindows()
                                time.sleep(1)  # Avoid busy waiting
                                continue

    except ValueError:
        print("Received:", line)

# Release the video capture and writer objects
if cap.isOpened():
    cap.release()
out.release()
cv2.destroyAllWindows()

# Close the serial port
ser.close()


Distance: 8 cm
Distance: 54 cm
Distance: 55 cm
Distance: 12 cm
Distance: 5 cm
Average Distance: 26.8 cm
Distance: 6 cm
Average Distance: 26.4 cm
Distance: 5 cm
Average Distance: 16.6 cm


c:\anaconda3\envs\torch\lib\site-packages\torch\nn\functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Sent: 1
Distance: 5 cm
Average Distance: 6.6 cm
Sent: 1
Distance: 54 cm
Average Distance: 15.0 cm
Sent: 1
Distance: 54 cm
Average Distance: 24.8 cm
Sent: 1
Distance: 155 cm
Average Distance: 54.6 cm
Distance: 54 cm
Average Distance: 64.4 cm
Distance: 54 cm
Average Distance: 74.2 cm
Distance: 55 cm
Average Distance: 74.4 cm
Distance: 155 cm
Average Distance: 94.6 cm
Distance: 55 cm
Average Distance: 74.6 cm
Distance: 155 cm
Average Distance: 94.8 cm
Distance: 55 cm
Average Distance: 95.0 cm
Distance: 54 cm
Average Distance: 94.8 cm
Distance: 54 cm
Average Distance: 74.6 cm
Distance: 54 cm
Average Distance: 74.4 cm
Distance: 54 cm
Average Distance: 54.2 cm
Distance: 55 cm
Average Distance: 54.2 cm
Distance: 155 cm
Average Distance: 74.4 cm
Distance: 155 cm
Average Distance: 94.6 cm
Distance: 54 cm
Average Distance: 94.6 cm
Distance: 55 cm
Average Distance: 94.8 cm
Distance: 55 cm
Average Distance: 94.8 cm
Distance: 54 cm
Average Distance: 74.6 cm
Distance: 54 cm
Average Distance: 54.4 cm